In [64]:
import os 

In [ ]:
%pwd

'c:\\Users\\panse\\Downloads\\Text summ project'

In [42]:
import os

target_directory ="c:\\Users\\panse\\Downloads\\Text summ project\\Text-summarization-Project-"
os.chdir(target_directory)


In [43]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [44]:
from Text_Summarizer.constants import *


In [45]:
from Text_Summarizer.utils.common import read_yaml , create_directories

In [47]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [48]:
import os
import urllib.request as request
import zipfile
from Text_Summarizer.logging import logger
from Text_Summarizer.utils.common import get_size

In [49]:
class DataIngestion:
    def __init__(self , config = DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
            url = self.config.source_URL,
            filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with the following info: \n{headers}")
        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok= True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [50]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e 

[2025-04-21 17:35:03,939: INFO: common: config\config.yaml loaded successfully]
[2025-04-21 17:35:03,947: INFO: common: params.yaml loaded successfully]
[2025-04-21 17:35:03,948: INFO: common: created directory at: artifacts]
[2025-04-21 17:35:03,949: INFO: common: created directory at: artifacts/data_ingestion]


[2025-04-21 17:35:07,838: INFO: 1725425473: artifacts/data_ingestion/data.zip download! with the following info: 
Connection: close
Content-Length: 8377572
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "3aea79eaa36efc9339c42d3a61413f75f62f7e5676002d7473efbe842605ace6"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 767D:256259:2ED679:86B33F:6806346E
Accept-Ranges: bytes
Date: Mon, 21 Apr 2025 12:05:05 GMT
Via: 1.1 varnish
X-Served-By: cache-del21740-DEL
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1745237104.440517,VS0,VE907
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: c940eee17acd3af6289f24b068e3f621965a1454
Expires: Mon, 21 Apr 2025 12:10:05 GMT
Source-Age: 0

]
